In [1]:
import bs4, tiktoken, numpy as np, os

from langchain import hub
from langchain.load import dumps, loads
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

from operator import itemgetter
from typing import Literal

from dotenv import load_dotenv
load_dotenv()

learn_api_key = os.environ['LANGCHAIN_API_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']
groq_api_key = os.environ['GROQ_API_KEY']

In [ ]:
# ### INDEXING ###
# loader = WebBaseLoader(
#     web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )

# blog_docs = loader.load()

# # Split
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)

# # Make Splits
# splits = text_splitter.split_documents(blog_docs)

# # Index
# vectorstore = Chroma.from_documents(
#     splits,
#     HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# )
# retriever = vectorstore.as_retriever()

### Logical Routing

In [2]:
# Data Model
class RouteQuery(BaseModel):
    """Route a user query to most relevant datasource."""
    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given the user question choose which datasource would be the most relevant for answering their question",
    )

# LLM with function call
llm = ChatGroq(model="gemma2-9b-it", api_key=groq_api_key, temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        {"role": "system", "content": system},
        {"role": "user", "content": "{question}"},
    ]
)

router = prompt | structured_llm

In [3]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french)"""

result = router.invoke({"question":question})
result

RouteQuery(datasource='python_docs')

In [4]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here
        return "chain for js_docs"
    else:
        ### Logic here
        return "golang_docs"
    
full_chain = router | RunnableLambda(choose_route)
full_chain.invoke({"question": question})

'chain for python_docs'

### Semantic Routing

In [5]:
# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering question about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute Similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = ({
    "query" : RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGroq(model="gemma2-9b-it", api_key=groq_api_key, temperature=0)
    | StrOutputParser()
)
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing, not even light, can escape. 

Imagine squeezing the entire mass of the sun into a space the size of a city. That's how incredibly dense a black hole is! This extreme density creates a gravitational pull so powerful that it warps the fabric of space and time around it. 

Anything that gets too close to a black hole gets pulled in and can never come out. We can't see black holes directly because they don't emit light, but we can detect them by observing their effects on nearby matter. 


Let me know if you have any other questions! 


